In [1]:
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re

table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
df = pd.read_parquet('metadata.parquet')

In [2]:
df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [3]:

# Our harm dictionary
# Organized as so: [HarmScore, Violent (0-1), Explicit (0-1)] 
harm_dict = {
    # Existing entries
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [5, 0, 1],
    "creampie": [6, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [10, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [5, 0, 1],
    "nsfw": [8, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "tits": [6, 0, 1],
    "milf": [8, 0, 1],

    # New or expanded terms
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "bukkake": [7, 1, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "double penetration": [8, 1, 1],  # or "dp"
    "ejaculation": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "incest": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "paizuri": [7, 0, 1],  # Japanese term (breast sex)
    "pedophilia": [10, 1, 1],
    "rape": [10, 1, 1],
    "rape fantasy": [10, 1, 1],
    "sadomasochism": [8, 1, 1],
    "scat": [9, 1, 1],  # feces-related fetish
    "snuff": [10, 1, 1], # sexual murder content
    "tentacle": [7, 0, 1],  # in anime/hentai context
    "underage porn": [10, 1, 1],
    "watersports": [8, 0, 1],  # urine fetish
    "zoophilia": [10, 1, 1]
}

innoTerms = {
    # Existing entries
    "woman": [2, 0, 0],
    "girl": [4, 0, 0],
    "teen": [4, 0, 0],
    "child": [4, 0, 0],
    "kid": [4, 0, 0],
    "teenager": [4, 0, 0],
    "boy": [4, 0, 0],
    "lolita": [10, 0, 0],

    # New or expanded terms
    "minor": [5, 0, 0],
    "preteen": [5, 0, 0],
    "underage": [6, 0, 0],
    "adolescent": [4, 0, 0],
    "young": [3, 0, 0],
    "juvenile": [4, 0, 0],
    "infant": [5, 0, 0],
    "toddler": [5, 0, 0],
    "schoolgirl": [5, 0, 0],
    "schoolboy": [5, 0, 0],
    "baby": [4, 0, 0]
}



def get_prompt_harm_score(prompt, harm_map, inno_map):
    tokens = prompt.lower().split()
    
    # 1) Check if there's at least one harm_dict token in the prompt
    has_harm = any(token in harm_map for token in tokens)
    
    # 2) If none found, ignore innoTerms and return 0
    if not has_harm:
        return 0
    
    # 3) Otherwise, sum up scores from BOTH dictionaries
    total_score = 0
    for token in tokens:
        # Harm dict score
        harm_score = harm_map.get(token, [0, 0, 0])[0]
        # Inno dict score
        inno_score = inno_map.get(token, [0, 0, 0])[0]
        
        total_score += harm_score
        total_score += inno_score
    
    return total_score



# Calculate harm scores for each prompt
df['harm_score'] = df['prompt'].apply(
    lambda x: get_prompt_harm_score(x, harm_dict, innoTerms)
)

print("Finished")


Finished


In [4]:
df['harm_score'].max()

30

In [5]:
print(df)

                                       image_name  \
0        2217ccbd-a1c6-47ac-9a2d-79649727c834.png   
1        c78807b7-d55a-4a2d-a6b6-9192b18941ad.png   
2        dc71658a-5e4b-4dca-861a-e1535510348b.png   
3        48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png   
4        601d9792-eccd-4850-97a7-edbe91d3464c.png   
...                                           ...   
1999995  4a34854e-e117-4b69-b1da-cc74534524c0.png   
1999996  d4ae6d4f-762c-42f0-9481-a1c233e6a541.png   
1999997  67dac228-3631-4036-8498-a40a67a7d354.png   
1999998  9d57179b-de12-460b-a699-a66dfca528be.png   
1999999  0ae508b1-f140-48c9-9da4-a124967215cf.png   

                                                    prompt  part_id  \
0        a portrait of a female robot made from code, v...        1   
1        a portrait of a female robot made from a cloud...        1   
2            only memories remain, trending on artstation         1   
3                         dream swimming pool with nobody         1   
4       

In [6]:
filteredData = df.sort_values(by="harm_score", ascending=False)

In [7]:
filteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
1404790,858045fb-7460-40df-81d5-db8fc499a7bf.png,1 6 k neon - monochrome 3 d of cgsociety : abs...,1405,3770620672,50,11.0,8,512,768,deleted_acount,2022-08-13 12:03:00+00:00,0.504399,0.052094,30
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,30
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,30
1404336,2acae79e-cb19-4750-8405-3c4f4de8ae9f.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1247897719,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 12:00:00+00:00,0.146162,0.131038,30
1404335,dd90c4b9-1d42-48cf-b005-f884f5d21055.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1205710592,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 11:48:00+00:00,0.900430,0.102768,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666849,639fc868-d1a4-4b46-8acf-de8e073d6664.png,"white mink with pink bacterial necklace , unre...",667,2163729190,50,7.0,8,512,512,cd0d504283e478af7a9ef88e6993fe76a2e895e12492c4...,2022-08-09 14:53:00+00:00,0.037813,0.119824,0
666848,8d7fae24-2e69-4532-95da-3d461584b2f2.png,"white mink with necklace of pink bacteria, unr...",667,4166410291,50,7.0,8,512,512,cd0d504283e478af7a9ef88e6993fe76a2e895e12492c4...,2022-08-09 14:52:00+00:00,0.063129,0.114815,0
666847,c2875e0e-c39a-43f2-bb43-029f3fc23367.png,"white mink with necklace of pink bacteria, unr...",667,3102154543,50,7.0,8,512,512,cd0d504283e478af7a9ef88e6993fe76a2e895e12492c4...,2022-08-09 14:51:00+00:00,0.040356,0.118560,0
666846,63006ff9-07d1-425d-9ab0-0447a1cde3a3.png,white mink with nimbus of pink bacteria around...,667,1392639471,50,7.0,8,512,512,cd0d504283e478af7a9ef88e6993fe76a2e895e12492c4...,2022-08-09 14:50:00+00:00,0.068155,0.124590,0


In [8]:
df['prompt_lower'] = df['prompt'].str.lower()

# 2) Check for harm_dict terms
terms_found_harm = []
for term in harm_dict.keys():
    pattern = rf"\b{re.escape(term)}\b"
    # Check if *any* row in 'prompt_lower' contains this term
    if df['prompt_lower'].str.contains(pattern, regex=True).any():
        terms_found_harm.append(term)

# 3) Check for innoTerms
terms_found_inno = []
for term in innoTerms.keys():
    pattern = rf"\b{re.escape(term)}\b"
    if df['prompt_lower'].str.contains(pattern, regex=True).any():
        terms_found_inno.append(term)

# 4) Print results
print("Harm dict terms found in the dataset:")
print(terms_found_harm)

print("\nInnoTerms found in the dataset:")
print(terms_found_inno)

Harm dict terms found in the dataset:
['porn', 'nude', 'erotic', 'bondage', 'boobs', 'creampie', 'fisting', 'gangbang', 'gore', 'hentai', 'moaning', 'nsfw', 'sex', 'sexual', 'shota', 'tits', 'milf', 'bukkake', 'cock', 'cumming', 'incest', 'rape', 'scat', 'snuff', 'tentacle']

InnoTerms found in the dataset:
['woman', 'girl', 'teen', 'child', 'kid', 'teenager', 'boy', 'lolita', 'minor', 'preteen', 'underage', 'adolescent', 'young', 'juvenile', 'infant', 'toddler', 'schoolgirl', 'schoolboy', 'baby']


In [8]:
df['harm_score'] = df['harm_score'] + 1

In [9]:
df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621,1
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707,1
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425,1
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586,1
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676,1
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115,1
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463,1
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547,1


## Muitiplication Factor(Term) / Total (Term[s]) 

### Scalability depending on the term T1, T2, T3, T4 Terms. If T1 term is combined with T2 Term = T4. Ex. "Lolita" & "T2" = Likely CP -> T4

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [11]:
df1 = df[:100000]
df2 = df[100001:200001]
df1.loc[df['harm_score'] >= 15, "harm_score"] = "Harmful"
df1.loc[df['harm_score']  < 15, "harm_score"] = "Harmless"
df2.loc[df['harm_score'] >= 15, "harm_score"] = "Harmful"
df2.loc[df['harm_score']  < 15, "harm_score"] = "Harmless"

C:\Users\JT\AppData\Local\Temp\ipykernel_10504\419697958.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Harmful' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.loc[df['harm_score'] >= 15, "harm_score"] = "Harmful"
C:\Users\JT\AppData\Local\Temp\ipykernel_10504\419697958.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Harmful' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df2.loc[df['harm_score'] >= 15, "harm_score"] = "Harmful"


In [15]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df1['prompt'])
y_train = df1['harm_score']

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=31)
knn.fit(X_train, y_train)

# Save model and vectorizer
joblib.dump(knn, "knn_moderation_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [16]:
accuracy_score(y, y_pred)

NameError: name 'y' is not defined

In [ ]:
precision_score(y, y_pred, average='macro')

In [ ]:
recall_score(y, y_pred, average='macro')

In [ ]:
f1_score(y, y_pred, average='macro')

In [61]:
accuracy_score(y, y_pred)

0.99999

In [62]:
precision_score(y, y_pred, average='macro')

C:\Users\JT\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.499995

In [63]:
recall_score(y, y_pred, average='macro')

0.5

In [64]:
f1_score(y, y_pred, average='macro')

0.4999974999874999

In [71]:
df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score,prompt_lower
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621,1,"a portrait of a female robot made from code, v..."
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707,1,a portrait of a female robot made from a cloud...
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425,1,"only memories remain, trending on artstation"
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586,1,dream swimming pool with nobody
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822,1,a dog doing weights. epic oil painting.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676,1,david bowie giving a piggy back ride to ziggy ...
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115,1,david bowie giving a piggy back ride to ziggy ...
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463,1,funny computer
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547,1,hilarious witty computing machine


In [14]:
import joblib


In [24]:
neigh = joblib.load("knn_moderation_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

def is_flagged(prompt):
    # Use transform(), NOT fit_transform()
    prompt_vectorized = vectorizer.transform([prompt])

    # Predict using the KNN model
    prediction = neigh.predict(prompt_vectorized)
    
    return prediction[0]  # Returns "Harmful" or "Harmless"

# Example usage
user_input = input("Enter a prompt: ")
print(f"Moderation result: {is_flagged(user_input)}")

Enter a prompt:  rape insect porn


Moderation result: Harmless
